In [30]:
import numpy as np; pi = np.pi
import scipy.io as spio
import sys
import bisect
from qutip import *

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import cm

In [31]:
# Python Standard Library
from itertools import starmap, product

def rho_to_pauli_basis(rho):
    """
    Given a quantum operator write it in
    vector form in the Pauli basis.
    """
    dim = rho.shape[0]
    nq = int(np.log2(dim))
    dims = [[2]*nq, [1]*nq]
    data = np.zeros((dim**2,1),dtype=np.complex64)
    pauli_basis = (qeye(2), sigmax(), sigmay(), sigmaz())
    for idx, op in enumerate(starmap(tensor,product(pauli_basis, repeat=nq))):
        data[idx] = (op*rho).tr() / np.sqrt(dim)
    return Qobj(data, dims=dims)

def pauli_basis_to_rho(rho):
    """
    Given a quantum operator in vector from that is written in the Pauli basis
    convert it to operator form in the computational basis.
    """
    dim = rho.shape[0]
    nq = int(np.log2(np.sqrt(dim)))
    dims = [[2]*nq, [2]*nq]
    data = rho.data.toarray()
    rho = 0
    pauli_basis = (qeye(2), sigmax(), sigmay(), sigmaz())
    for idx, op in enumerate(starmap(tensor,product(pauli_basis, repeat=nq))):
        rho += (op*complex(data[idx]))*1/np.sqrt(2*nq)
    return Qobj(rho, dims=dims)

In [32]:
num_lvl = 20 # number of levels
a = destroy(num_lvl) # annihilation operator
K = 1 # kerr amplitude
G = 4*K # two photon pump amplitude
alpha = np.sqrt(G/K) # coherent state amplitude

# cat states
cat_plus = (coherent(num_lvl,alpha) + coherent(num_lvl,-alpha)).unit()
cat_minus = (coherent(num_lvl,alpha) - coherent(num_lvl,-alpha)).unit()

# computational basis
up = (cat_plus + cat_minus)/np.sqrt(2)
down = (cat_plus - cat_minus)/np.sqrt(2)

# Identity in computational basis
I = up*up.dag() + down*down.dag()

# sigma-z in computational basis
sigma_z = up*up.dag() - down*down.dag()

# sigma-x in computational basis
sigma_x = up*down.dag() + down*up.dag()

# sigma-y in computational basis
sigma_y = 1j*(-up*down.dag() + down*up.dag())

# Array with Pauli matrices
P = [I, sigma_x, sigma_y, sigma_z]

In [33]:
eye = qeye(num_lvl) # identity operator
a1 = tensor([a,eye])
a2 = tensor([eye,a])

# sigma z
sigma_z1 = tensor([sigma_z,eye])
sigma_z2 = tensor([eye,sigma_z])

# initial state
psi0 = tensor([up+down,up+down]).unit()

# coupling
g = 0.1

# gate time
T_g = lambda Theta : Theta/(4*g*pow(alpha,2))
    
# Hamiltonian
H1 = - K * pow(a1.dag(),2)*pow(a1,2) + G * (pow(a1.dag(),2) + pow(a1,2))
H2 =  - K * pow(a2.dag(),2)*pow(a2,2) + G * (pow(a2.dag(),2) + pow(a2,2))
H_coupling = a1.dag()*a2 + a2.dag()*a1
H_tot = [H_coupling]

In [34]:
# Pauli matrices
d = 4
Q = []
for i in range(d):
    Q.extend([tensor(P[i], P[j]) for j in range(d)])

In [21]:
Theta = 2*pi/(3600) # one half of a degree
gamma = 1/1500 # single-photon loss rate
c_ops = [np.sqrt(gamma)*a1,np.sqrt(gamma)*a2] # collapse operator
# For precise calculation
opt = Options(nsteps=15e3)
# pauli transfer matrix
R = np.zeros((d**2,d**2))

for j in range(d**2):
    rho = mesolve(H_tot, Q[j], [0,T_g(Theta)] , c_ops, options = opt)
    Lambda = rho.states[-1]
    for i in range(d**2):
        R[i,j] = 1/d * np.real((Q[i]*Lambda).tr())
R = Qobj(R,dims=[[2,2] for i in range(2)]) # Make quantum object

In [22]:
R

Quantum object: dims = [[2, 2], [2, 2]], shape = (16, 16), type = oper, isherm = False
Qobj data =
[[ 9.99990481e-01  2.87670437e-08  0.00000000e+00  0.00000000e+00
   2.87670437e-08 -3.36477529e-11  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00 -2.68675889e-10
   0.00000000e+00  0.00000000e+00 -2.68675890e-10  0.00000000e+00]
 [ 3.26717832e-08  9.99832364e-01  0.00000000e+00  0.00000000e+00
   6.59906928e-13  2.87671358e-08  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  1.96523936e-09
   0.00000000e+00  0.00000000e+00 -1.74521273e-02  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  9.99832364e-01  0.00000000e+00
   0.00000000e+00  0.00000000e+00  2.87672572e-08  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   3.19551843e-10  1.74521270e-02  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  9.99990481e-01
   0.00000000e+00  0.00000000e+00  0.000

In [26]:
phi = pi
ket0 = basis(2,0)
ket1 = basis(2,1)
psi = tensor([(ket0+ket1).unit() for i in range(2)])
rho = ket2dm(psi) # initial rho
rho_vec = rho_to_pauli_basis(rho)
U = R
for i in range(round(phi/Theta)):
    U = U*R
rho_vec_final = U*rho_vec
rho_final = pauli_basis_to_rho(rho_vec_final)
rho_final

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[0.24575049+0.j         0.24315346-0.00423821j 0.24315346-0.00423821j
  0.24065423+0.j        ]
 [0.24315346+0.00423821j 0.24575049+0.j         0.24065423+0.j
  0.24315346+0.00423821j]
 [0.24315346+0.00423821j 0.24065423+0.j         0.24575049+0.j
  0.24315346+0.00423821j]
 [0.24065423+0.j         0.24315346-0.00423821j 0.24315346-0.00423821j
  0.24575049+0.j        ]]

In [19]:
tensor(rho_final,qeye(4))

Quantum object: dims = [[2, 2, 4], [2, 2, 4]], shape = (16, 16), type = oper, isherm = True
Qobj data =
[[0.24575049+0.j         0.        +0.j         0.        +0.j
  0.        +0.j         0.24315346-0.00423821j 0.        +0.j
  0.        +0.j         0.        +0.j         0.24315346-0.00423821j
  0.        +0.j         0.        +0.j         0.        +0.j
  0.24065423+0.j         0.        +0.j         0.        +0.j
  0.        +0.j        ]
 [0.        +0.j         0.24575049+0.j         0.        +0.j
  0.        +0.j         0.        +0.j         0.24315346-0.00423821j
  0.        +0.j         0.        +0.j         0.        +0.j
  0.24315346-0.00423821j 0.        +0.j         0.        +0.j
  0.        +0.j         0.24065423+0.j         0.        +0.j
  0.        +0.j        ]
 [0.        +0.j         0.        +0.j         0.24575049+0.j
  0.        +0.j         0.        +0.j         0.        +0.j
  0.24315346-0.00423821j 0.        +0.j         0.        +0.j
  0.     

In [28]:
gate_expand_2toN(rho_final,4,targets=[0,1])

Quantum object: dims = [[2, 2, 2, 2], [2, 2, 2, 2]], shape = (16, 16), type = oper, isherm = True
Qobj data =
[[0.24575049+0.j         0.        +0.j         0.        +0.j
  0.        +0.j         0.24315346-0.00423821j 0.        +0.j
  0.        +0.j         0.        +0.j         0.24315346-0.00423821j
  0.        +0.j         0.        +0.j         0.        +0.j
  0.24065423+0.j         0.        +0.j         0.        +0.j
  0.        +0.j        ]
 [0.        +0.j         0.24575049+0.j         0.        +0.j
  0.        +0.j         0.        +0.j         0.24315346-0.00423821j
  0.        +0.j         0.        +0.j         0.        +0.j
  0.24315346-0.00423821j 0.        +0.j         0.        +0.j
  0.        +0.j         0.24065423+0.j         0.        +0.j
  0.        +0.j        ]
 [0.        +0.j         0.        +0.j         0.24575049+0.j
  0.        +0.j         0.        +0.j         0.        +0.j
  0.24315346-0.00423821j 0.        +0.j         0.        +0.j
  0

In [66]:
def carb(theta):
    zz = tensor(sigmaz(),sigmaz())
    return (-1j*theta/2*zz).expm()

In [67]:
target_state = carb(phi)*ket2dm(psi)
target_state

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = False
Qobj data =
[[0.-0.25j 0.-0.25j 0.-0.25j 0.-0.25j]
 [0.+0.25j 0.+0.25j 0.+0.25j 0.+0.25j]
 [0.+0.25j 0.+0.25j 0.+0.25j 0.+0.25j]
 [0.-0.25j 0.-0.25j 0.-0.25j 0.-0.25j]]

In [72]:
fidelity(rho_final,target_state)

0.09206742439793833